In [2]:
using MPSKit, MPSKitModels, TensorKit, TensorOperations

In [3]:
function bosonicOperators(cutoff::Int, elt=Float64)
    creadat = zeros(elt, cutoff + 1, cutoff + 1)

    for i in 1:cutoff
        creadat[i + 1, i] = sqrt(i)
    end

    a⁺ = TensorMap(creadat, ℝ^(cutoff + 1), ℝ^(cutoff + 1))
    a⁻ = TensorMap(collect(creadat'), ℝ^(cutoff + 1), ℝ^(cutoff + 1))
    return (a⁺, a⁻)
end

bosonicOperators (generic function with 2 methods)

In [4]:
# define Hamiltionian for infinite basis with chemical potential
function BoseHubbardHamiltonian(t, U, μ, L, M)

    a⁺, a⁻ = bosonicOperators(M) # bosonic operators
    n_op = a⁺ * a⁻

    H_i = TensorMap(zeros, Float64, (ℝ^(M+1))^2 ← (ℝ^(M+1))^2)
    H_i += -t * (a⁺ ⊗ a⁻ + a⁻ ⊗ a⁺) + U/2 * n_op * (n_op - id(ℝ^(M+1))) ⊗ id(ℝ^(M+1))
    H_i += -μ * n_op ⊗ id(ℝ^(M+1))

    return @mpoham sum(H_i{i,i+1} for i in (1:L))
end

BoseHubbardHamiltonian (generic function with 1 method)

In [8]:
H = BoseHubbardHamiltonian(0.1, 2, 0, 12, 5)

MPOHamiltonian{CartesianSpace, TensorMap{CartesianSpace, 2, 2, Trivial, Matrix{Float64}, Nothing, Nothing}, Float64}(MPSKit.SparseMPOSlice{CartesianSpace, TensorMap{CartesianSpace, 2, 2, Trivial, Matrix{Float64}, Nothing, Nothing}, Float64}[[TensorMap((ℝ^1 ⊗ ℝ^6) ← (ℝ^6 ⊗ ℝ^1)):
[:, :, 1, 1] =
 1.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  1.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  1.0  0.0  0.0  0.0

[:, :, 4, 1] =
 0.0  0.0  0.0  1.0  0.0  0.0

[:, :, 5, 1] =
 0.0  0.0  0.0  0.0  1.0  0.0

[:, :, 6, 1] =
 0.0  0.0  0.0  0.0  0.0  1.0
 TensorMap((ℝ^1 ⊗ ℝ^6) ← (ℝ^6 ⊗ ℝ^3)):
[:, :, 1, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 2, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0

[:, :, 3, 1] =
 0.0  0.0  -0.08276058886023684  0.0  0.0  0.0

[:, :, 4, 1] =
 0.0  0.0  0.0  -0.24828176658071027  0.0  0.0

[:, :, 5, 1] =
 0.0  0.0  0.0  0.0  -0.4965635331614207  0.0

[:, :, 6, 1] =
 0.0  0.0  0.0  0.0  0.0  -0.8276058886023683

[:, :, 1, 2] =
 -2.867980228534952e-17  5.647435290633063e-20  0

In [9]:
state = FiniteMPS(rand, Float64, 12, ℝ^(5+1), ℝ^10)

FiniteMPS{TensorMap{CartesianSpace, 2, 1, Trivial, Matrix{Float64}, Nothing, Nothing}, TensorMap{CartesianSpace, 1, 1, Trivial, Matrix{Float64}, Nothing, Nothing}}(Union{Missing, TensorMap{CartesianSpace, 2, 1, Trivial, Matrix{Float64}, Nothing, Nothing}}[TensorMap((ℝ^1 ⊗ ℝ^6) ← ProductSpace(ℝ^6)):
[:, :, 1] =
 0.2505379720069365  0.18939680372434586  …  0.2724668548274079

[:, :, 2] =
 0.17471561796082014  -0.07527232772159476  …  0.8112752503128298

[:, :, 3] =
 0.31756884347682923  -0.30447229077563925  …  -0.48461127789759667

[:, :, 4] =
 0.4761963532643415  0.6601099151692729  …  -0.17641005566425721

[:, :, 5] =
 -0.7251053547397923  0.48722627954101144  …  -0.0400245553237113

[:, :, 6] =
 0.23089925236221848  0.4388906515158967  …  -0.00487009409014294
, TensorMap((ℝ^6 ⊗ ℝ^6) ← ProductSpace(ℝ^10)):
[:, :, 1] =
  0.3140435465135685    0.4693143190313848     …  0.43256952494183964
  0.0433891427035464    0.08713646250845797       0.10178795966454267
  0.03217376439287138   0.039

In [10]:
(groundstate,_) = find_groundstate(state, H, DMRG(tol=1e-10, maxiter=1000));

┌ Info: Iteraton 0 error 0.8073038228206745
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 1 error 0.011804761811585712
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 2 error 0.0012282199558998853
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 3 error 0.0008722561011261707
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 4 error 0.000822456195368803
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


...

┌ Info: Iteraton 394 error 1.6237216920526547e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 395 error 1.4802359849693461e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27
┌ Info: Iteraton 396 error 1.3494093713824824e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 397 error 1.230163516289061e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 398 error 1.121438926127747e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27
┌ Info: Iteraton 399 error 1.0223653212626134e-10
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


┌ Info: Iteraton 400 error 9.32011739784879e-11
└ @ MPSKit /root/.julia/packages/MPSKit/90CTP/src/algorithms/groundstate/dmrg.jl:27


In [11]:
E0 = sum(expectation_value(groundstate, H))

-0.9239100553549678

In [12]:
a⁺, a⁻ = bosonicOperators(5) # bosonic operators
n_op = a⁺ * a⁻

TensorMap(ProductSpace(ℝ^6) ← ProductSpace(ℝ^6)):
 0.0  0.0  0.0                 0.0                 0.0  0.0
 0.0  1.0  0.0                 0.0                 0.0  0.0
 0.0  0.0  2.0000000000000004  0.0                 0.0  0.0
 0.0  0.0  0.0                 2.9999999999999996  0.0  0.0
 0.0  0.0  0.0                 0.0                 4.0  0.0
 0.0  0.0  0.0                 0.0                 0.0  5.000000000000001


In [13]:
N_b = expectation_value(groundstate, n_op)

12-element Vector{Float64}:
 0.4993125000929445
 0.5287431067385662
 0.5444728793981843
 0.519560943340981
 0.5482028102975521
 0.5178605630208389
 0.5487691090604397
 0.5177185003002037
 0.5487619661629333
 0.5175138156811749
 0.5713971031284198
 4.137660533372673

In [14]:
sum(N_b)

9.99997383059491